In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import numpy.ma as ma
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.ticker as ticker
import matplotlib.dates as dates
from mpl_toolkits.axes_grid1 import ImageGrid,make_axes_locatable,host_subplot
#from mpl_toolkits.basemap import Basemap
from datetime import datetime, timedelta
import sys
import os
import pyPIPS.utils as utils
import pyPIPS.thermolib as thermo
import pyPIPS.DSDlib as dsd
#import pyPIPS.disdrometer_module as dis
import pyPIPS.plotmodule as PIPSplot
#import pyPIPS.simulator as sim
import pyPIPS.pips_io as pipsio
import pyPIPS.PIPS as pips
import pyPIPS.parsivel_params as pp
import pyPIPS.parsivel_qc as pqc
import pyPIPS.radarmodule as radar
#from pyCRMtools.modules import plotmodule as plotmod
from pyCRMtools.modules import utils as CRMutils
from pyCRMtools.pycaps import arps_read
from pyCRMtools.pycaps import pycaps_fields
from pyCRMtools.pycaps import calvars_radar as pycaps_radar
import pandas as pd
import xarray as xr
import glob
import numpy.random as random
from scipy.stats import gamma, uniform
from scipy.special import gamma as gammafunc
from scipy import ndimage
from metpy.plots import StationPlot
from metpy.calc import wind_components
from metpy.cbook import get_test_data
from metpy.plots import StationPlot
from metpy.plots.wx_symbols import current_weather, sky_cover
from metpy.units import units
%matplotlib inline

In [ ]:
from natsort import natsorted
radar_name = 'KHTX'
radar_dir = '/Users/dawson29/sshfs_mounts/depot/data/Projects/VORTEXSE/obsdata/2017/NEXRAD/PIPS2A_FMCW/0430/CFRadial/modified/'
radar_paths = glob.glob(radar_dir + '/cfrad.*')
radar_paths = natsorted(radar_paths)

In [ ]:
sweeptime_start = radar._getsweeptime(radar_paths[0])
sweeptime_stop = radar._getsweeptime(radar_paths[-1])
print(sweeptime_start, sweeptime_stop)

In [ ]:
starttime = '20170430234917'
stoptime = '20170430234917'

radar_dict = radar.read_sweeps(radar_paths, starttime, stoptime)
print(radar_dict['radarsweeplist'])

In [ ]:
print(radar_dict['radarsweeplist'][0].info())

In [ ]:
import pyart
import cartopy.crs as ccrs

for radar_obj in radar_dict['radarsweeplist'][:1]:
    
    xrad, yrad, zrad = radar_obj.get_gate_x_y_z(0)
    display = pyart.graph.RadarMapDisplay(radar_obj)
    fig = plt.figure(figsize=(10, 8))

    projection = ccrs.LambertConformal(central_latitude=radar_obj.latitude['data'][0],
                                       central_longitude=radar_obj.longitude['data'][0])
    
#     projection = ccrs.LambertConformal(central_latitude=PIPS_lat,
#                                        central_longitude=PIPS_lon)
    # plot D0
    ax = fig.add_subplot(111)
    print("Time: ", radar_obj.metadata['start_datetime'])
    print("Elevation: ", radar_obj.elevation['data'][-1])
    display.plot_ppi_map('D0', 0, title='D0', cmap='viridis',
                         vmin=0., vmax=4., colorbar_label='', ax=ax, resolution='10m', projection=projection,
                         fig=fig)
                         # min_lon=-88., max_lon=-86., min_lat=40., max_lat=41.)
    # plot ZDR
    
#     display.plot('ZDR', 0, title='ZDR',
#                  vmin=0, vmax=6, colorbar_label='', ax=ax)
    display.plot_range_ring(radar_obj.range['data'][-1]/1000., ax=ax)
    #display.set_limits(xlim=(-50, 50), ylim=(-50, 50), ax=ax)
    #display.set_limits(xlim=(-75, -55), ylim=(75, 95), ax=ax)
    #display.plot_point(PIPS_lon, PIPS_lat, 'r*', ms=20)
    display.ax.set_extent([-60000, 60000, -60000, 60000], crs=projection)
    #ax.plot(PIPS_rad_loc[0]/1000., PIPS_rad_loc[1]/1000., 'r*', ms=20)
    #ax.plot(xrad[theta_index, range_index]/1000., yrad[theta_index, range_index]/1000., 'rx')
    #ax.contourf(xrad / 1000., yrad / 1000., distance)
    #ax.text(PIPS_rad_loc[0] / 1000. - 5., PIPS_rad_loc[1] / 1000., 'PIPS1A', fontsize=10, bbox=dict(facecolor='white', alpha=0.5))
    plt.show()